In [2]:
def compara(ai,aj,max=True):
  # Higher is better.
  if (max):
    res = 0
    if (ai > aj):
      res = 1
  # Lower is better.    
  else:
    res = 1
    if (ai > aj):
      res = 0

  return res  

# Cálculo de phi+ para 1 critério (coluna).
def tabelaC(df,crit,min):
  tc1 = []
  for i in df.index:
    ai = df.loc[i,crit]
    aux = []
    # Comparação de ai com vários aj.
    for j in df.index:
      aj = df.loc[j,crit]
      if (min):  
        res = compara(ai,aj,False)
      else:
        res = compara(ai,aj,True)  
      aux.append(res)
    tc1.append(aux)

  return tc1


# Cálculo de phi- para 1 critério (coluna).
def tabelaCM(df,crit,min):
  tc1 = []
  for i in df.index:
    ai = df.loc[i,crit]
    aux = []
    # Comparação de ai com vários aj.
    for j in df.index:
      aj = df.loc[j,crit]
      if (min):  
        res = compara(aj,ai,False)
      else:
        res = compara(aj,ai,True)  
      aux.append(res)
    tc1.append(aux)

  return tc1

# Cálculo do Phi+ para todos os critérios (colunas).
def calcPhipPesos(df,pesos,objetivos):

  # Obtendo o número de alternativas (linhas da tabela).
  rows = df.index
  m = len(rows)
  # Obtendo os critérios (colunas da tabela).
  columns = df.columns
  n = len(columns) 

  #-----------------------------------------------------------
  # Cálculo do Phi+ (todas as colunas).
  #-----------------------------------------------------------
  # Comparando as alternativas em relação ao primeiro critério.
  tc = tabelaC(df,columns[0],objetivos[0]) 
  dc = pesos[0]*pd.DataFrame(data=tc)
  #print("Cálculo do critério "+columns[0]) 
 
  # Comparando as alternativas nos demais critérios e armazenado
  # o resultado acumulado em dc.
  for i in range(1,n):
    tci = tabelaC(df,columns[i],objetivos[i])
    dci = pesos[i]*pd.DataFrame(data=tci)
    #print("Cálculo do critério "+columns[i])
    dc = dc.add(dci, fill_value=0)

  # Resultando final: para cada linha soma das colunas.
  dc.loc[:,'Total'] = dc.sum(axis = 1, skipna = True)
  # Dividir por 1/(m-1) a soma total.
  dc['Total'] = (dc['Total']/(m-1)).round(2)
  #-----------------------------------------------------------

  return dc['Total']


# Cálculo do Phi- para todos os critérios (colunas).
def calcPhinPesos(df,pesos,objetivos):

  # Obtendo o número de alternativas (linhas da tabela).
  rows = df.index
  m = len(rows)
  # Obtendo os critérios (colunas da tabela).
  columns = df.columns
  n = len(columns) 

  #-----------------------------------------------------------
  # Cálculo do Phi- (todas as colunas).
  #-----------------------------------------------------------
  # Comparando as alternativas em relação ao primeiro critério.
  tc2 = tabelaCM(df,columns[0],objetivos[0]) 
  dc2 = pesos[0]*pd.DataFrame(data=tc2)
  #print("Cálculo do critério "+columns[0]) 
 
  # Comparando as alternativas nos demais critérios e armazenado
  # o resultado acumulado em dc.
  for i in range(1,n):
    tci2 = tabelaCM(df,columns[i],objetivos[i])
    dci2 = pesos[i]*pd.DataFrame(data=tci2)
    #print("Cálculo do critério "+columns[i])
    dc2 = dc2.add(dci2, fill_value=0)

  # Resultando final: para cada linha soma das colunas.
  dc2.loc[:,'Total'] = dc2.sum(axis = 1, skipna = True)
  # Dividir por 1/(m-1) a soma total.
  dc2['Total'] = (dc2['Total']/(m-1)).round(2)

  return dc2['Total']



# Cálculo do índice de ranqueamento do Promethee II.
def prometheeIIpesos(df,pesos,objetivos):
  #print("Iniciando cálculos do Promethee II...")  

  # Cálculo do Phi+.
  #print('Cálculos do Phi+...')
  dcp = calcPhipPesos(df,pesos,objetivos)
  
  # Cálculo do Phi-.
  #print('Cálculos do Phi-...')
  dcn = calcPhinPesos(df,pesos,objetivos)

  #print('Cálculos do Phi = Phi+ - Phi-...')

  return dcp-dcn

In [3]:
import pandas as pd

#pd.set_option('display.max_rows', None)

pesos = [0.2, 0.3, 0.5, 0.3, 0.3, 0.4, 0.2, 0.4, 0.4, 0.5, 0.5, 1]

objetivos = [False, False, False, False, False, False, False, False, False, False, False, False]

df = pd.read_csv('subdim_only_numbers.csv')

In [4]:
phi = prometheeIIpesos(df,pesos,objetivos)
df.loc[:,'Phi']= list(phi)
df

,0.292111732,0.709370632,0.844012334,0.571428571,0.003483054,0,0.5,0.695181259,0.404761905,0.439765106,0.491474746,0.06779201,Phi
0,0.294667,0.623339,0.801920,0.000000,0.000657,0.000000,0.000000,0.533664,0.374048,0.413952,0.352418,0.067792,-0.83
1,0.282476,0.594759,0.771491,0.000000,0.001220,0.002002,0.000000,0.326012,0.400238,0.344832,0.500990,0.067792,-0.75
2,0.270997,0.671739,0.826817,0.000000,0.000258,0.000000,0.083333,0.787747,0.367143,0.424545,0.474748,0.067792,0.49
3,0.435084,0.647601,0.791001,0.000000,0.000610,0.000200,0.250000,0.662081,0.362302,0.400696,0.461352,0.067792,0.28
4,0.388350,0.654867,0.819654,0.000000,0.000023,0.000000,0.750000,0.642609,0.542143,0.398232,0.419054,0.067792,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,0.293506,0.654161,0.816772,0.000000,0.000141,0.000000,0.250000,0.584959,0.388889,0.431379,0.374342,0.141344,0.95
640,0.238031,0.598016,0.774520,0.000000,0.000000,0.000000,0.166667,0.204784,0.463333,0.314446,0.385984,0.067792,-2.11
641,0.213479,0.631283,0.852068,0.571429,0.005219,0.001601,0.333333,0.808548,0.506825,0.393765,0.523447,0.109060,2.97
642,0.288423,0.705293,0.859427,1.000000,0.004949,0.000600,0.166667,0.881524,0.545476,0.434597,0.501349,0.085599,3.43


In [5]:

df['Phi']

0     -0.83
1     -0.75
2      0.49
3      0.28
4      0.26
       ... 
639    0.95
640   -2.11
641    2.97
642    3.43
643   -0.47
Name: Phi, Length: 644, dtype: float64